### Intents.json – The data file which has predefined patterns and responses.
### train_chatbot.py – A script to build the model and train our chatbot.
### Words.pkl – Contains a list of our vocabulary.
### Classes.pkl – Contains the list of categories.
### Chatbot_model.h5 – Trained model (with model info like weights of the neurons).
### Chatgui.py – Script implementing GUI for our chatbot. Users can easily interact with the bot.


## Import and load the data file

The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt') #needed for tokenizing classes (preprocessing)
nltk.download('wordnet') #needed for lemmatizing
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
print(intents) #nice

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-09-26 19:44:00.924611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-26 19:44:00.924701: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

## Preprocess data

Tokenizing text data (basic first step) also create a list of classes for our tags.

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### lemmatize each word and remove duplicate words from the list
#### lemmatize = sort words by grouping inflected or variant forms of the same word. nltk handles it

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create training and testing data

    Create the training data, we will provide the input and the output.
    Input will be the pattern
    Output will be the class that input pattern belongs to
    Computer doesn’t understand text so we will convert text into numbers.

In [4]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object) #we are using tuples in the array so use object bc immutableness
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Build the model

    We have our training data ready
    now we will build a deep neural network that has 3 layers. 
    use the Keras sequential API 

In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/1000
10/10 [==============================] - 0s 1ms/step - loss: 2.2477 - accuracy: 0.0638
Epoch 2/1000
10/10 [==============================] - 0s 2ms/step - loss: 2.0625 - accuracy: 0.2766
Epoch 3/1000
10/10 [==============================] - 0s 2ms/step - loss: 2.1437 - accuracy: 0.1915
Epoch 4/1000
10/10 [==============================] - 0s 1ms/step - loss: 1.9626 - accuracy: 0.2979
Epoch 5/1000
10/10 [==============================] - 0s 2ms/step - loss: 1.8312 - accuracy: 0.4043
Epoch 6/1000
10/10 [==============================] - 0s 1ms/step - loss: 1.7378 - accuracy: 0.5106
Epoch 7/1000
10/10 [==============================] - 0s 2ms/step - loss: 1.6051 - accuracy: 0.5957
Epoch 8/1000
10/10 [==============================] - 0s 1ms/step - loss: 1.5422 - accuracy: 0.5106
Epoch 9/1000
10/10 [==============================] - 0s 2ms/step - loss: 1.3725 - accuracy: 0.5319
Epoch 10/1000
10/10 [==============================] - 0s 1ms/step - loss: 1.2083 - accuracy: 0.6809

10/10 [==============================] - 0s 2ms/step - loss: 0.0494 - accuracy: 1.0000
Epoch 82/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0496 - accuracy: 1.0000
Epoch 83/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0750 - accuracy: 0.9787
Epoch 84/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0553 - accuracy: 0.9787
Epoch 85/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0544 - accuracy: 1.0000
Epoch 86/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 87/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 88/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 89/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0303 - accuracy: 1.0000
Epoch 90/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0112 - accuracy: 1.0000
Epoc

10/10 [==============================] - 0s 1ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 162/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 163/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0540 - accuracy: 0.9787
Epoch 164/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 165/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 166/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0184 - accuracy: 1.0000
Epoch 167/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 168/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 169/1000
10/10 [==============================] - 0s 977us/step - loss: 0.0343 - accuracy: 1.0000
Epoch 170/1000
10/10 [==============================] - 0s 980us/step - loss: 0.0125 - accuracy

10/10 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 242/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 243/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 244/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 245/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 246/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 247/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0222 - accuracy: 1.0000
Epoch 248/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 249/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 250/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0090 - accuracy: 1.

10/10 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 322/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 323/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0273 - accuracy: 0.9787
Epoch 324/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 325/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 326/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 327/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 328/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 329/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 330/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 1.

...

10/10 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 640/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 641/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0119 - accuracy: 1.0000
Epoch 642/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 643/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 644/1000
10/10 [==============================] - 0s 1ms/step - loss: 8.9051e-04 - accuracy: 1.0000
Epoch 645/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 646/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 647/1000
10/10 [==============================] - 0s 1ms/step - loss: 9.1846e-04 - accuracy: 1.0000
Epoch 648/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0095 - accu

10/10 [==============================] - 0s 2ms/step - loss: 0.0414 - accuracy: 0.9787
Epoch 719/1000
10/10 [==============================] - 0s 1ms/step - loss: 4.4127e-04 - accuracy: 1.0000
Epoch 720/1000
10/10 [==============================] - 0s 1ms/step - loss: 6.9661e-04 - accuracy: 1.0000
Epoch 721/1000
10/10 [==============================] - 0s 1ms/step - loss: 8.7581e-04 - accuracy: 1.0000
Epoch 722/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 723/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 724/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 725/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 726/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 727/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0094 - 

10/10 [==============================] - 0s 1ms/step - loss: 2.9391e-04 - accuracy: 1.0000
Epoch 798/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 799/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 800/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 801/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 802/1000
10/10 [==============================] - 0s 880us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 803/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 804/1000
10/10 [==============================] - 0s 2ms/step - loss: 5.9185e-04 - accuracy: 1.0000
Epoch 805/1000
10/10 [==============================] - 0s 1ms/step - loss: 9.3333e-04 - accuracy: 1.0000
Epoch 806/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0027 

10/10 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 877/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 878/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 879/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 880/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 881/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 882/1000
10/10 [==============================] - 0s 1ms/step - loss: 4.1852e-04 - accuracy: 1.0000
Epoch 883/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 884/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 885/1000
10/10 [==============================] - 0s 870us/step - loss: 0.0012 - accura

10/10 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 956/1000
10/10 [==============================] - 0s 1ms/step - loss: 9.4927e-04 - accuracy: 1.0000
Epoch 957/1000
10/10 [==============================] - 0s 2ms/step - loss: 4.3304e-04 - accuracy: 1.0000
Epoch 958/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 959/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 960/1000
10/10 [==============================] - 0s 2ms/step - loss: 7.3680e-04 - accuracy: 1.0000
Epoch 961/1000
10/10 [==============================] - 0s 2ms/step - loss: 8.3927e-04 - accuracy: 1.0000
Epoch 962/1000
10/10 [==============================] - 0s 1ms/step - loss: 6.3725e-04 - accuracy: 1.0000
Epoch 963/1000
10/10 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 964/1000
10/10 [==============================] - 0s 1ms/step - loss: 0

### 200 epochs later, the accuracy regularly tests at 100% (on training data)


## Predict the response 

    We will load the trained model, then use a GUI to predict the response from the bot. 
    model only tells us the class it belongs to
    so we implement some functions to identify the class, then retrieve a random response from the list of responses.

    import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [8]:
#already made as .py's
%run chatApp.py
%run chatGUI.py